In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams.update({'font.size': 22})

In [ ]:
def read_nested_lists_from_file(filename):
    nested_lists = []
    current_list = []

    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespace
            if line:  # If the line is not empty
                # Split the line into items and convert to appropriate types
                items = [eval(item) if item.replace('.', '', 1).isdigit() else item for item in line.split(', ')]
                current_list.append(items)
            else:  # Empty line indicates the end of a list
                if current_list:
                    nested_lists.append(current_list)
                    current_list = []  # Reset for the next list

    # Add any remaining list
    if current_list:
        nested_lists.append(current_list)

    return nested_lists

In [ ]:
def moving_average(data, window_size):
    data = np.array(data).astype(float)
    return np.convolve(data, np.ones(window_size) / window_size, mode='valid')

In [ ]:
# Data loading

factor = 10

losses_data = read_nested_lists_from_file(f"../Data/train_losses_logress_{factor}x.txt")
exptest_data = read_nested_lists_from_file("../Data/train_losses_logress_algo.txt")
adadelta_data = read_nested_lists_from_file("../Data/train_losses_logress_adadelta.txt")

train_losses_adam_list = losses_data[0]
train_losses_exptest_list = exptest_data[0]
train_losses_sgd_momentum_list = losses_data[1]
train_losses_sgd_list = losses_data[2]
train_losses_rmsprop_list = losses_data[3]
train_losses_adadelta_list = adadelta_data[0]

test_losses_data = read_nested_lists_from_file(f"../Data/test_losses_logress_{factor}x.txt")[0]
test_loss_exptest_data = read_nested_lists_from_file("../Data/test_losses_logress_algo.txt")[0]
test_loss_adadelta_data = read_nested_lists_from_file("../Data/test_losses_logress_adadelta.txt")[0]

test_accuracy_adam_list = test_losses_data[0]
test_accuracy_exptest_list = test_loss_exptest_data[0]
test_accuracy_sgd_momentum_list = test_losses_data[1]
test_accuracy_sgd_list = test_losses_data[2]
test_accuracy_rmsprop_list = test_losses_data[3]
test_accuracy_adadelta_list = test_loss_adadelta_data[0]

print(f"LR = {factor} * max_lr")
print("Mean +/- STD of Test Accuracies (Adam):", round(100*np.mean(test_accuracy_adam_list), 2), "+/-", round(100*np.std(test_accuracy_adam_list), 4))
print("Mean +/- STD of Test Accuracies (ExpTest):", round(100*np.mean(test_accuracy_exptest_list), 2), "+/-", round(100*np.std(test_accuracy_exptest_list), 4))
print("Mean +/- STD of Test Accuracies (SGD with Momentum):", round(100*np.mean(test_accuracy_sgd_momentum_list), 2), "+/-", round(100*np.std(test_accuracy_sgd_momentum_list), 4))
print("Mean +/- STD of Test Accuracies (SGD):", round(100*np.mean(test_accuracy_sgd_list), 2), "+/-", round(100*np.std(test_accuracy_sgd_list), 4))
print("Mean +/- STD of Test Accuracies (RMSprop):", round(100*np.mean(test_accuracy_rmsprop_list), 2), "+/-", round(100*np.std(test_accuracy_rmsprop_list), 4))
print("Mean +/- STD of Test Accuracies (Adadelta):", round(100*np.mean(test_accuracy_adadelta_list), 2), "+/-", round(100*np.std(test_accuracy_adadelta_list), 4))

LR = 10 * max_lr
Mean +/- STD of Test Accuracies (Adam): 88.3 +/- 1.5545
Mean +/- STD of Test Accuracies (ExpTest): 92.3 +/- 0.0729
Mean +/- STD of Test Accuracies (SGD with Momentum): 87.62 +/- 1.2462
Mean +/- STD of Test Accuracies (SGD): 87.96 +/- 2.6518
Mean +/- STD of Test Accuracies (RMSprop): 88.71 +/- 1.0121
Mean +/- STD of Test Accuracies (Adadelta): 91.46 +/- 0.354


In [ ]:
# Plotting

colors = {
    "Adam": "b",
    "ExpTest": "g",
    "SGD w/ Mom.": "r",
    "SGD": "c",
    "RMSprop": "m",
    "Adadelta": "k",
}

# Comment out optimizers that do not appear on graph
optimizers = {
    # "Adam": train_losses_adam_list,
    "ExpTest": train_losses_exptest_list,
    # "SGD w/ Mom.": train_losses_sgd_momentum_list,
    # "SGD": train_losses_sgd_list,
    # "RMSprop": train_losses_rmsprop_list,
    "Adadelta": train_losses_adadelta_list,
}

window_size = round(60000 / 32)

plt.figure(figsize=(12, 8))

for optimizer_name, train_losses_list in optimizers.items():
    losses_array = np.array([moving_average(train_loss, window_size) for train_loss in train_losses_list])
    mean_loss = np.mean(losses_array, axis=0)
    sem_loss = np.std(losses_array, axis=0) / np.sqrt(losses_array.shape[0])
    iters = np.linspace(1, 5, num=len(mean_loss))

    plt.plot(iters, mean_loss, label=optimizer_name, color=colors[optimizer_name])
    plt.fill_between(iters, mean_loss - sem_loss, mean_loss + sem_loss, alpha=0.4, color=colors[optimizer_name])

plt.xlabel('Epochs')
plt.ylabel('Training Loss')
plt.ylim(top=0.4, bottom=0.25)
leg = plt.legend()
for legobj in leg.legend_handles:
    legobj.set_linewidth(4)
plt.grid()
plt.savefig(f"MNIST_{factor}.png", dpi=600)
plt.show()